In [1]:
import pickle
import os

In [2]:
def load_pickle(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(f)

In [3]:
Upitt_csv2wiki = load_pickle("./Dataset/Upitt/csv2wiki.pickle")
Upitt_wiki2csv = load_pickle("./Dataset/Upitt/wiki2csv.pickle")
Upitt_csv_mapping = load_pickle("./Dataset/Upitt/csv_mapping_id.pickle")
Upitt_wiki_mapping = load_pickle("./Dataset/Upitt/wiki_mapping_id.pickle")


ST_csv2wiki = load_pickle("./Dataset/Standford/csv2wiki.pickle")
ST_wiki2csv = load_pickle("./Dataset/Standford/wiki2csv.pickle")
ST_csv_mapping = load_pickle("./Dataset/Standford/csv_mapping_id.pickle")
ST_wiki_mapping = load_pickle("./Dataset/Standford/wiki_mapping_id.pickle")


In [4]:
def integrate_mapping(target1, target2):
    total_mapping =  set(list(target1) + list(target2))
    total_mapping = {value : idx for idx, value in enumerate(total_mapping)}
    return total_mapping


In [5]:
total_csv_mapping_id = integrate_mapping(Upitt_csv_mapping.keys(), ST_csv_mapping.keys())

In [6]:
total_wiki_mapping_id = integrate_mapping(Upitt_wiki_mapping.keys(), ST_wiki_mapping.keys())

In [7]:
# rerpot
print(f"Before_num_wikis = {len(Upitt_wiki_mapping) + len(ST_wiki_mapping)}")
print(f"Current_num_wikis = {len(total_wiki_mapping_id)}")

print(f"Before_num_csv = {len(Upitt_csv_mapping) + len(ST_csv_mapping)}")
print(f"Current_num_csv = {len(total_csv_mapping_id)}")

Before_num_wikis = 2657
Current_num_wikis = 2512
Before_num_csv = 77
Current_num_csv = 77


In [8]:
ST_wiki_mapping_r = {v:k for k,v in ST_wiki_mapping.items()}
Upitt_wiki_mapping_r = {v:k for k,v in Upitt_wiki_mapping.items()}

In [9]:
def update_new_csv_id(ST_csv_mapping, ST_wiki_mapping_r, ST_csv2wiki,
                      csv_mapping, wiki_mapping):
    update_dict = {}
    for b_key, b_value in ST_csv_mapping.items():
        
        # update
        new_key_id = csv_mapping[b_key]
        b_wikis = ST_csv2wiki[b_value]
        new_wikis_id = list(map(lambda x: wiki_mapping[ST_wiki_mapping_r[x]], b_wikis))
        
        # save
        update_dict[new_key_id] = new_wikis_id
    
    return update_dict

In [10]:
new_ST_csv2wiki = update_new_csv_id(ST_csv_mapping, ST_wiki_mapping_r, ST_csv2wiki,
                                    total_csv_mapping_id, total_wiki_mapping_id)

In [11]:
new_Upitt_csv2wiki = update_new_csv_id(Upitt_csv_mapping, Upitt_wiki_mapping_r, Upitt_csv2wiki,
                                       total_csv_mapping_id, total_wiki_mapping_id)

In [12]:
new_ST_csv2wiki.update(new_Upitt_csv2wiki)
total_csv2wiki = new_ST_csv2wiki

In [13]:
def get_new_wiki2csv(new_csv2wiki):
    new_wiki2csv = {}
    for csv, wikis in new_csv2wiki.items():
        for wiki in wikis:
            try:
                new_wiki2csv[wiki].append(csv)
            except:
                new_wiki2csv[wiki] = [csv]
    return new_wiki2csv

In [15]:
total_wiki2csv = get_new_wiki2csv(total_csv2wiki)

In [16]:
print(f"num_new_csv2wiki = {len(total_csv2wiki)}")
print(f"num_new_wiki2csv = {len(total_wiki2csv)}")

num_new_csv2wiki = 77
num_new_wiki2csv = 2512


In [18]:
names = ["total_wiki_mapping_id","total_csv_mapping_id","total_csv2wiki","total_wiki2csv"]

base_path = f'./Dataset/Total'
if not os.path.exists(base_path):
    os.makedirs(base_path)

for name in names:
    with open(f'{base_path}/{name[6:]}.pickle','wb') as f:
        pickle.dump(eval(name), f, pickle.HIGHEST_PROTOCOL)